# Urban PointCloud Suspended Street Light Extractor

This notebook shows a "complete solution" in which a single point cloud file is automatically labeled using the various tools available in this repository. For clarity we skip [preprocessing of AHN data](1.%20AHN%20preprocessing.ipynb) and assume all necassary data files are already available.

We build a `Pipeline` with different `Processors`, each of which labels a particular type of object. The result is a labelled pointcloud, where labels are stored in the LAS extra_dim `label`.

The `Pipeline` supports processing a single file, or batch-processing a folder.

In [ ]:
# Add project src to path.
import set_path

# Import modules.
import logging

import src.utils.ahn_utils as ahn_utils
import src.utils.bgt_utils as bgt_utils
import src.utils.plot_utils as plot_utils
import src.utils.log_utils as log_utils
from src.pipeline import Pipeline
from src.labels import Labels

from src.stages.building_filter import BuildingFilter
from src.stages.vertical_segmentation_filter import LowHeightFilter, HighHeightFilter
from src.stages.cable_extractor import CableExtractor
from src.stages.tramcable_classifier import TramCableClassifier
from src.stages.streetlight_detector import StreetlightDetector

In [ ]:
# Set-up logging.
logfile = 'pipeline.log'
log_utils.reset_logger()

# INFO messages will be printed to console.
log_utils.add_console_logger(logging.DEBUG)
# All messages will be printed to a file. Use `clear_log` flag to clear the log file, if desired.
log_utils.add_file_logger(logfile, clear_log=True)

In [ ]:
# Set-up data files.

folder = '../demo_dataset'

# AHN data folder.
ahn_data_folder = folder + '/ahn/'
# File with BGT building polygons.
bag_building_file = folder + '/bag/bag_buildings.csv'
# File with BGT tramtrack linestrings.
bgt_tramtrack_file = folder + '/bgt/bgt_tram_tracks.csv'

# Initialize AHN reader for elevation data.
ahn_reader = ahn_utils.NPZReader(ahn_data_folder)

# Initialize BGT readers for BGT polygon and point data.
bag_buidling_reader = bgt_utils.BGTPolyReader(bgt_file=bag_building_file)
bgt_tramtrack_reader = bgt_utils.BGTLineReader(bgt_file=bgt_tramtrack_file)

---
## Data Fusion

### Search Space Reduction
First, we use data fusion to automatically label ground and building points and reduce the dataset.

In [ ]:
# Ground fuser
ground_params = {'min_point_height':4.5}
ground_fuser = LowHeightFilter(Labels.GROUND, ahn_reader=ahn_reader, **ground_params)

# Sky fuser
sky_params = {'max_point_height':12.5}
sky_fuser = HighHeightFilter(Labels.SKY, ahn_reader=ahn_reader, **sky_params)

# Building fuser
bld_params = {'building_offset':1.25}
building_fuser = BuildingFilter(Labels.BUILDING, ahn_reader=ahn_reader, bgt_reader=bag_buidling_reader, **bld_params)

### Cable Segments
Cable points meeting certain conditions are labelled as such.

In [ ]:
# Cable Extractor
cable_params = {}
cable_extractor = CableExtractor(Labels.CABLE, **cable_params)

### Cable Classification

Then we label cables beloning to the tram cable network by the BGT tram data.

In [ ]:
# Tram cable Extractor
tram_params = {'min_height':7.5}
tram_cable_extractor = TramCableClassifier(Labels.TRAM_CABLE, ahn_reader=ahn_reader, bgt_reader=bgt_tramtrack_reader, **tram_params)

### Armatuur Extractor

Next, we look for 'armaturen' suspended street lights.

In [ ]:
# Armatuur Extractor
armatuur_params = {
    'width': (.2, 1),
    'height': (.15, 1),
    #'z_offset':(0,.55),
    'axis_offset': 0.2
}
armatuur_extractor = StreetlightDetector(Labels.ARMATUUR, armatuur_params=armatuur_params, min_cable_bending=2)

---
## Set-up the pipeline

In [ ]:
# Set-up pipeline.
process_sequence = (building_fuser, ground_fuser, sky_fuser, cable_extractor, tram_cable_extractor, armatuur_extractor)
pipeline = Pipeline(processors=process_sequence, ahn_reader=ahn_reader, caching=True)

---
## Process a single file

In [ ]:
log_utils.reset_logger()
log_utils.add_console_logger(logging.DEBUG)

In [ ]:
# Select the file to process.
tile_code = '2352_9744'
in_file = folder + '/pointcloud/labelled_' + tile_code + '.laz'
out_file = folder + '/pointcloud/processed_' + tile_code + '.laz'

In [ ]:
pipeline.process_file(in_file, out_file=out_file, in_labels=False)

---
## Process a folder

It is also possible to process all LAS files in a folder in one batch.

In [ ]:
# Re-set logging to suppress console output.
log_utils.reset_logger()
# Only ERROR messages will be printed to console.
log_utils.add_console_logger(level=logging.ERROR)
# All messages will be printed to a file. Use `clear_log` flag to clear the log file, if desired.
log_utils.add_file_logger(logfile, clear_log=True)

In [ ]:
pipeline.process_folder(in_folder=folder+'/pointcloud/', in_prefix='labelled_', out_prefix='processed_', in_labels=False)

---
## View the result

To get a quick look at the result, we can plot a slice of the labelled point cloud at a specified height.

In [ ]:
%matplotlib widget
plot_utils.plot_bgt_bag_and_cloudslice(tile_code, out_file, ahn_reader, building_file=bag_building_file,
                                   tram_file=bgt_tramtrack_file, min_plane_height=4.5, max_plane_height=12, hide_noise=True)